In [1]:
import pandas as pd
import numpy as np
import os
from metrics import compute_our_bias_metrics_for_model

In [2]:
MODEL_LIST = ['cnn','roberta-base-unbiased','roberta-base-unbiased-small','kaggle_bert','my_kaggle_bert']

In [3]:
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
TOXICITY_COLUMN = 'target'

In [4]:
def convert_to_bool(df, col_name):
    df[col_name] = np.where(df[col_name] >= 0.5, True, False)
    
def convert_dataframe_to_bool(df, MODEL_NAME):
    bool_df = df.copy()
    for col in ['target', MODEL_NAME] + identity_columns: # no target here
        convert_to_bool(bool_df, col)
    return bool_df

Demographic Parity and Equalized Opportunity

In [5]:
results = {}
for MODEL_NAME in MODEL_LIST:
    if not os.path.exists(f'./bias/{MODEL_NAME}_bias.csv'):
        eval_df = pd.read_csv(f'./submissions/{MODEL_NAME}_submission.csv')
        eval_df = convert_dataframe_to_bool(eval_df, MODEL_NAME)
        our_bias_df = compute_our_bias_metrics_for_model(eval_df, identity_columns, MODEL_NAME, TOXICITY_COLUMN)
        results[MODEL_NAME] = our_bias_df
        our_bias_df.to_csv(f'./bias/{MODEL_NAME}_bias.csv', index=False)
    else:
        our_bias_df = pd.read_csv(f'./bias/{MODEL_NAME}_bias.csv')
        results[MODEL_NAME] = our_bias_df

In [6]:
model_res = results['cnn']
model_res

,subgroup,subgroup_size,demographics_parity,equalized_opportunity
0,christian,2109,0.035562,0.280952
1,female,2602,0.051499,0.317143
2,male,2112,0.068655,0.343750
3,jewish,411,0.070560,0.308824
4,muslim,1054,0.081594,0.270386
5,homosexual_gay_or_lesbian,538,0.104089,0.227273
6,psychiatric_or_mental_illness,238,0.126050,0.462963
7,white,1178,0.140917,0.345609
8,black,761,0.143233,0.333333


In [7]:
val = model_res[model_res['subgroup']=='female']['demographics_parity']
val.values[0]

0.0514988470407378

In [8]:
genders = ['male', 'female', 'homosexual_gay_or_lesbian']
religions = ['christian', 'jewish', 'muslim']
races = ['black', 'white']
illnesses = ['psychiatric_or_mental_illness']
groups = [genders, religions, races, illnesses]
groups_name = ['genders', 'religions', 'races', 'illnesses']

### Get Pairwise Ratio

In [9]:
demo_pairity_dict = {}
equal_opportunity_dict = {}

In [10]:
for metric in ['demographics_parity','equalized_opportunity']:
    for model_name, model_res in results.items():
        print('\n\n=============',model_name,'=============\n\n')
        ratios = []
        for idx, group in enumerate(groups): # for all sensitive groups
            score_dict = {} # dict for storing scores
            for subgroup in group:
                val = model_res[model_res['subgroup']==subgroup][metric].values[0]
                score_dict[subgroup] = val
            if len(group) <= 1: # for illneses
                print(groups_name[idx])
                print('---------')
                print(f'{group[0]}: {score_dict[group[0]]:.4f}')
                demo_pairity_dict[(model_name, groups_name[idx])] = (score_dict[group[0]],score_dict[group[0]])
                equal_opportunity_dict[(model_name, groups_name[idx])] = (score_dict[group[0]],score_dict[group[0]])
                continue
            for i in range(len(group)):
                for j in range(i+1,len(group)):
                    if score_dict[group[i]] < score_dict[group[j]]:
                        temp = score_dict[group[i]]/score_dict[group[j]]
                        ratios.append((group[i],group[j], temp))
                    else:
                        temp = score_dict[group[j]]/score_dict[group[i]]
                        ratios.append((group[j],group[i], temp))
            max_diff = float('-inf')
            min_diff = float('inf')

            for key1, value1 in score_dict.items():
                for key2, value2 in score_dict.items():
                    if key1 != key2:
                        diff = abs(value1 - value2)
                        if diff > max_diff:
                            max_diff = diff
                            max_diff_keys = (key1, key2)
                        if diff < min_diff:
                            min_diff = diff
                            min_diff_keys = (key1, key2)

            # print(max_diff_keys, min_diff_keys)
            # print(max_diff, min_diff)
            smallest_ratio = min(ratios, key=lambda x: x[2])
            largest_ratio = max(ratios, key=lambda x: x[2])
            print(groups_name[idx])
            print('--------')
            print(f'max difference: {max_diff_keys} | {max_diff:.4f}')
            print(f'min difference: {min_diff_keys} | {min_diff:.4f}')
            print(f'smallest ratio: {smallest_ratio}')
            print(f'largest ratio: {largest_ratio}')
            print()
            if metric == 'demographics_parity':
                demo_pairity_dict[(model_name, groups_name[idx])] = (smallest_ratio[2], largest_ratio[2])
            elif metric == 'equalized_opportunity':
                equal_opportunity_dict[(model_name, groups_name[idx])] = (smallest_ratio[2], largest_ratio[2])





============= cnn =============


genders
--------
max difference: ('female', 'homosexual_gay_or_lesbian') | 0.0526
min difference: ('male', 'female') | 0.0172
smallest ratio: ('female', 'homosexual_gay_or_lesbian', 0.4947567804985168)
largest ratio: ('female', 'male', 0.7501073444830227)

religions
--------
max difference: ('christian', 'muslim') | 0.0460
min difference: ('jewish', 'muslim') | 0.0110
smallest ratio: ('christian', 'muslim', 0.43583975652519097)
largest ratio: ('jewish', 'muslim', 0.8647654614383528)

races
--------
max difference: ('black', 'white') | 0.0023
min difference: ('black', 'white') | 0.0023
smallest ratio: ('christian', 'muslim', 0.43583975652519097)
largest ratio: ('white', 'black', 0.9838320275385117)

illnesses
---------
psychiatric_or_mental_illness: 0.1261


============= roberta-base-unbiased =============


genders
--------
max difference: ('female', 'homosexual_gay_or_lesbian') | 0.0852
min difference: ('male', 'female') | 0.0244
smallest ratio: ('

### Compare models bias in different groups


#### Demographic parity


In [11]:
max_len = len('roberta-base-unbiased-small')
for idx, group in enumerate(groups):
    print(f'\n======== {groups_name[idx]} =======\n')
    spaces = 3
    print(f"{' ' * max_len}\tsmallest ratio{'':>{spaces}}|{'':>{spaces}}largest ratio")
    for model_name in MODEL_LIST:
        print(f'{model_name.ljust(max_len)}: ', end='')
        print(f'{str(demo_pairity_dict[(model_name, groups_name[idx])]):<}')


======== genders =======

                           	smallest ratio   |   largest ratio
cnn                        : (0.4947567804985168, 0.7501073444830227)
roberta-base-unbiased      : (0.4729430058222235, 0.7583332431174286)
roberta-base-unbiased-small: (0.36183704842428877, 0.6669919459947196)
kaggle_bert                : (0.5068240190472617, 0.7768968925002747)
my_kaggle_bert             : (0.4812610988894485, 0.8147007306361423)

======== religions =======

                           	smallest ratio   |   largest ratio
cnn                        : (0.43583975652519097, 0.8647654614383528)
roberta-base-unbiased      : (0.4729430058222235, 0.8307460333778828)
roberta-base-unbiased-small: (0.36183704842428877, 0.6669919459947196)
kaggle_bert                : (0.4204354730670516, 0.8548256285482564)
my_kaggle_bert             : (0.4486508039139616, 0.8147007306361423)

======== races =======

                           	smallest ratio   |   largest ratio
cnn                        

#### Equalized Opportunity

In [12]:
max_len = len('roberta-base-unbiased-small')
for idx, group in enumerate(groups):
    print(f'\n======== {groups_name[idx]} =======\n')
    spaces = 3
    print(f"{' ' * max_len}\tsmallest ratio{'':>{spaces}}|{'':>{spaces}}largest ratio")
    for model_name in MODEL_LIST:
        print(f'{model_name.ljust(max_len)}: ', end='')
        print(f'{str(equal_opportunity_dict[(model_name, groups_name[idx])]):<}')


======== genders =======

                           	smallest ratio   |   largest ratio
cnn                        : (0.6611570247933882, 0.9225974025974025)
roberta-base-unbiased      : (0.8102589234664707, 0.9315363881401616)
roberta-base-unbiased-small: (0.8661654135338344, 0.959975696817802)
kaggle_bert                : (0.8363963080944213, 0.9717879604672057)
my_kaggle_bert             : (0.848575508261372, 0.9621540762902019)

======== religions =======

                           	smallest ratio   |   largest ratio
cnn                        : (0.6611570247933882, 0.9623917945733618)
roberta-base-unbiased      : (0.8102589234664707, 0.9894346087949745)
roberta-base-unbiased-small: (0.8248910675381262, 0.959975696817802)
kaggle_bert                : (0.8363963080944213, 0.9861146175208281)
my_kaggle_bert             : (0.848575508261372, 0.9737124463519312)

======== races =======

                           	smallest ratio   |   largest ratio
cnn                        : (0.66